In [2]:
import os
import numpy as np
from Bio.PDB import PDBParser, Superimposer

def extract_chain_segment_atoms(structure, chain_id, start, end):
    atoms = []
    for model in structure:
        for chain in model:
            if chain.id == chain_id:
                for residue in chain:
                    if start <= residue.id[1] <= end:
                        for atom in residue:
                            atoms.append(atom)
    return atoms

def calculate_rmsd(pdb1, pdb2, chain_id, start, end):
    parser = PDBParser(QUIET=True)
    structure1 = parser.get_structure('struct1', pdb1)
    structure2 = parser.get_structure('struct2', pdb2)
    
    atoms1 = extract_chain_segment_atoms(structure1, chain_id, start, end)
    atoms2 = extract_chain_segment_atoms(structure2, chain_id, start, end)

    if len(atoms1) != len(atoms2):
        raise ValueError("Segments do not have the same number of atoms")

    sup = Superimposer()
    sup.set_atoms(atoms1, atoms2)
    
    return sup.rms

def calculate_diversity(directory, chain_id, start, end):
    pdb_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdb') and not f.startswith('REF') and not f.endswith('_patch.pdb')]
    num_files = len(pdb_files)
    rmsd_matrix = np.zeros((num_files, num_files))
    
    for i in range(num_files):
        for j in range(i + 1, num_files):
            rmsd = calculate_rmsd(pdb_files[i], pdb_files[j], chain_id, start, end)
            rmsd_matrix[i, j] = rmsd
            rmsd_matrix[j, i] = rmsd  # The matrix is symmetric
    print('\n'.join(pdb_files))

    upper_triangle_indices = np.triu_indices(num_files, k=1)
    upper_triangle_values = rmsd_matrix[upper_triangle_indices]
    diversity = np.mean(upper_triangle_values)
    
    return diversity

# Example usage
directory = '/hanchenchen/ab_opt/AbDesign/test_results_20240725/codesign_single/0-5xku_C_B_A/H_CDR3'
chain_id = 'C'
start = 95
end = 102
diversity = calculate_diversity(directory, chain_id, start, end)
print(f"Averaged RMSD: {diversity}")

Averaged RMSD: 0.16103325415773315


In [3]:
import glob
import tqdm
import pandas as pd
import json

pdb_list = []
diversity_list = []
for directory in tqdm.tqdm(glob.glob('/hanchenchen/ab_opt/AbDesign/test_results/codesign_single/*/H_CDR3')):
    metadata_path = directory.replace('H_CDR3', 'metadata.json')
    metadata = json.load(open(metadata_path))
    assert len(metadata['items']) == 1
    chain_id = metadata['items'][0]["residue_first"][0]
    start = metadata['items'][0]["residue_first"][1]
    end = metadata['items'][0]["residue_last"][1]
    diversity = calculate_diversity(directory, chain_id, start, end)
    pdb_list.append(directory.split('/')[-2])
    diversity_list.append(diversity)

df = pd.DataFrame({'pdb': pdb_list, 'averaged_rmsd': diversity_list})
df

100%|██████████| 19/19 [01:06<00:00,  3.50s/it]


,pdb,averaged_rmsd
0,4-5tlk_B_A_X,0.209235
1,7-5w9h_H_I_G,0.352446
2,10-7bwj_H_L_E,0.189009
3,18-5tlk_D_C_X,0.096993
4,17-7che_A_B_R,0.116880
5,8-5tlj_B_A_X,0.329004
6,13-8ds5_C_B_A,0.068601
7,5-5tlj_D_C_X,0.066502
8,12-5w9h_B_C_A,0.109900
9,2-7chf_H_L_R,0.038997


In [5]:
df.to_csv('/hanchenchen/ab_opt/rebuttal_exps/2_diversity/results/averaged_upper_triangle_rmsd.csv', index=False)